<a href="https://colab.research.google.com/github/Ryzon3/csci_6967/blob/main/homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>